In [ ]:
!pip install pyAudioAnalysis
!pip install huggingface_hub
!pip install pyannote.audio
!pip install pydub
!apt-get install ffmpeg


In [ ]:
from huggingface_hub import login
from pyannote.audio import Pipeline
from pydub import AudioSegment

# Login using your Hugging Face API token
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Initialize the pre-trained speaker diarization pipeline
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization", use_auth_token="...")


pipeline.to(device)

# Upload your audio file
# from google.colab import files
# uploaded = files.upload()

# audio_file = next(iter(uploaded.keys()))

audio_file =  "/content/5Nafar.wav"
# Perform speaker diarization
diarization = pipeline(audio_file)

# Load the audio file using pydub for splitting
audio = AudioSegment.from_wav(audio_file)

# Create a dictionary to store segments by speaker
speaker_segments = {}

# Iterate over diarization segments and save each speaker's segments
for segment, track, speaker in diarization.itertracks(yield_label=True):
    start_time = segment.start * 1000  # Convert to milliseconds for pydub
    end_time = segment.end * 1000

    # Extract the speaker's audio segment
    speaker_audio = audio[start_time:end_time]

    # Append the segment to the corresponding speaker's list
    if speaker not in speaker_segments:
        speaker_segments[speaker] = []
    speaker_segments[speaker].append(speaker_audio)

# Save segments for each speaker
for speaker, segments in speaker_segments.items():
    combined_audio = AudioSegment.empty()
    for segment in segments:
        combined_audio += segment
    combined_audio.export(f"{speaker}.wav", format="wav")

    print(f"Saved {speaker}.wav with {len(segments)} segments.")
